### Print Directory Tree

In [25]:
import os
startpath = "F:\\Downloads\\train\\"
def list_files(startpath):
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print('{}{}/'.format(indent, os.path.basename(root)))
        subindent = ' ' * 4 * (level + 1)
        i = 0
        for f in files:
            i += 1
            print('{}{}'.format(subindent, f))
            if i >=3:
                break

list_files(startpath)

/
    first_try.h5
    KERAS_DOG_CAT.ipynb
.ipynb_checkpoints/
    KERAS_DOG_CAT-checkpoint.ipynb
data/
    train/
        cats/
            cat.0.jpg
            cat.1.jpg
            cat.10.jpg
        dogs/
            dog.0.jpg
            dog.1.jpg
            dog.10.jpg
    validation/
        cats/
            cat.1000.jpg
            cat.1001.jpg
            cat.1002.jpg
        dogs/
            dog.1001.jpg
            dog.1002.jpg
            dog.1003.jpg
preview/
    cat_0_1079.jpeg
    cat_0_1129.jpeg
    cat_0_1146.jpeg


### Test Data Augmentation Function (ImageDataGenerator)

In [5]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

img = load_img('data/train/cats/cat.0.jpg')  # this is a PIL image
x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

# the .flow() command below generates batches of randomly transformed images
# and saves the results to the `preview/` directory
i = 0
for batch in datagen.flow(x, batch_size=1,
                          save_to_dir='preview', save_prefix='cat', save_format='jpeg'):
    i += 1
    if i > 20:
        break  # otherwise the generator would loop indefinitely

### Define Model

In [12]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
img_width, img_height = 150, 150

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)
    
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [14]:
batch_size = 16

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        'data/train',  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        'data/validation',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')


Found 2001 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


### Train Model

In [24]:
model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=10,
        validation_data=validation_generator,
        validation_steps=800 // batch_size)
model.save_weights('first_try.h5')  # always save your weights after training or during training

Epoch 1/10
125/125 [==============================] - 115s - loss: 0.7059 - acc: 0.5665 - val_loss: 0.6561 - val_acc: 0.6550
Epoch 2/10
125/125 [==============================] - 117s - loss: 0.6489 - acc: 0.6195 - val_loss: 0.6296 - val_acc: 0.6525
Epoch 3/10
125/125 [==============================] - 123s - loss: 0.6210 - acc: 0.6570 - val_loss: 0.6021 - val_acc: 0.6613
Epoch 4/10
125/125 [==============================] - 127s - loss: 0.6064 - acc: 0.6805 - val_loss: 0.5763 - val_acc: 0.6837
Epoch 5/10
125/125 [==============================] - 121s - loss: 0.5923 - acc: 0.6935 - val_loss: 0.6146 - val_acc: 0.6450
Epoch 6/10
125/125 [==============================] - 116s - loss: 0.5585 - acc: 0.7170 - val_loss: 0.5776 - val_acc: 0.6763
Epoch 7/10
125/125 [==============================] - 116s - loss: 0.5679 - acc: 0.7275 - val_loss: 0.6420 - val_acc: 0.6763
Epoch 8/10
125/125 [==============================] - 114s - loss: 0.5488 - acc: 0.7475 - val_loss: 0.5434 - val_acc: 0.7275
